In [1]:
import os
import json
import re
import pandas as pd

# Pfad zum Hauptverzeichnis mit den Unterverzeichnissen
BASE_DIR = "data/liboqs"

def extract_model_name(directory_name):
    """Extrahiert den Modelnamen aus dem Verzeichnisnamen."""
    match = re.match(r"testresult_liboqs_(.*?)_\d{4}\.\d{2}\.\d{2}_\d{2}\.\d{2}", directory_name)
    return match.group(1) if match else directory_name

# Lade die JSON-Datei
def load_kem_memory_data(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

# Extrahiere relevante Daten aus der JSON-Datei
def parse_kem_memory_data(data, model_name):
    records = []
    for algorithm, details in data.items():
        if algorithm in ["cpuinfo", "config"]:
            continue  # Überspringe Meta-Daten

        keygen = details.get("keygen", {})
        encaps = details.get("sign", {})
        decaps = details.get("verify", {})

        records.append({
            "Model": model_name,
            "Algorithmus": algorithm,
            "maxHeap_key": keygen.get("maxHeap", 0),
            "maxHeap_enc": encaps.get("maxHeap", 0),
            "maxHeap_dec": decaps.get("maxHeap", 0),
            "maxStack_key": keygen.get("maxStack", 0),
            "maxStack_enc": encaps.get("maxStack", 0),
            "maxStack_dec": decaps.get("maxStack", 0)
        })
    
    return records

# Hauptfunktion zum Laden und Verarbeiten der Daten
def main():
    all_records = []
    
    for directory in os.listdir(BASE_DIR):
        full_path = os.path.join(BASE_DIR, directory)
        if os.path.isdir(full_path):
            json_file = os.path.join(full_path, "test_sig_mem_fullHeap.json")
            if os.path.isfile(json_file):
                model_name = extract_model_name(directory)
                data = load_kem_memory_data(json_file)
                recs = parse_kem_memory_data(data, model_name)
                all_records.extend(recs)
    
    # DataFrame erstellen
    df = pd.DataFrame(all_records)
    df = df.sort_values(by=["Algorithmus", "Model"])
    
    # Speichere als CSV
    df.to_csv("output/csv/liboqs_mem_sig_fullHeap.csv", index=False)
    
    # Anzeige im Jupyter Notebook
    from IPython.display import display
    display(df)
    
    return df

# Starte die Verarbeitung
df = main()


,Model,Algorithmus,maxHeap_key,maxHeap_enc,maxHeap_dec,maxStack_key,maxStack_enc,maxStack_dec
47,Pi1b,Dilithium2,5881856,5881856,5881856,0,0,0
0,Pi1bPlus,Dilithium2,5881856,5881856,5881856,0,0,0
94,Pi3b,Dilithium2,7249920,7249920,7249920,0,0,0
141,Pi4b,Dilithium2,7249920,7249920,7249920,0,0,0
188,PiBanana,Dilithium2,9306112,9306112,9306112,0,0,0
...,...,...,...,...,...,...,...,...
46,Pi1bPlus,cross-rsdpg-256-small,5881856,5881856,5881856,0,0,0
140,Pi3b,cross-rsdpg-256-small,7249920,7249920,7249920,0,0,0
187,Pi4b,cross-rsdpg-256-small,7249920,7249920,7249920,0,0,0
234,PiBanana,cross-rsdpg-256-small,9306112,9306112,9306112,0,0,0
